<a href="https://colab.research.google.com/github/RussBurnham/BookRecommendationWithKNN/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-01-02 01:43:22--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   151MB/s    in 0.2s    

2024-01-02 01:43:22 (151 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
ratings_per_user = df_ratings['user'].value_counts()
ratings_per_book = df_ratings['isbn'].value_counts()

users_to_keep = ratings_per_user[ratings_per_user >= 200].index
books_to_keep = ratings_per_book[ratings_per_book >= 100].index

filtered_ratings = df_ratings[
    (df_ratings['user'].isin(users_to_keep)) &
    (df_ratings['isbn'].isin(books_to_keep))
]

pivot_ratings = filtered_ratings.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

sparse_ratings_matrix = csr_matrix(pivot_ratings.values)

knn = NearestNeighbors(n_neighbors=6, metric='cosine')
knn.fit(sparse_ratings_matrix)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [52]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    k = 5
    matching_books = df_books[df_books['title'].str.lower() == book.lower()]
    if not matching_books.empty:
        book_id = matching_books.iloc[0]['isbn']
        book_index = pivot_ratings.index.get_loc(book_id)
        query_matrix = sparse_ratings_matrix[book_index]
        distances, indices = knn.kneighbors(query_matrix, n_neighbors=k+1)
        recommended_books = [
            (df_books[df_books['isbn'] == pivot_ratings.index[indices.flatten()[i+1]]]['title'].values[0], round(distances.flatten()[i+1], 2))
            for i in range(k)
        ]
        recommended_books = recommended_books[::-1]
        recommended_books = recommended_books[:-1]
        return [book, recommended_books]
    else:
        return None

In [54]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [("I'll Be Seeing You", 0.8), ('The Weight of Water', 0.77), ('The Surgeon', 0.77), ('I Know This Much Is True', 0.77)]]
You passed the challenge! 🎉🎉🎉🎉🎉
